# Dependences

In [ ]:
include("../../src/struct_data.jl")
include("../../src/neighbor.jl")
include("../../src/forces/forces.jl")

# Model & Aggregate

In [ ]:
@time Model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 100000.0,
        dt    = 0.5,
        nₖₙₙ  = 101,
        nₛₐᵥₑ = 500
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_1",
        path_output = ""
    )
)
dump(Model)

In [ ]:
Par1, Par2 = (0.9,2.0,4.0), 0.1

FusionAGG = nothing
# @time @start_agg FusionAGG = FusionAggregate(
@time FusionAGG = FusionAggregate(
    [
        AggType(
            "HEK_1", 
            InteractionPar(
                Cubic(Par1...), 
                ContractilePar(Par2)
            ),
            Float64.(readdlm("../../data/init/Sphere/10.0.xyz")[3:end,2:end]) |> cu
        )
    ], 
    Model
)

# kNN & Force

In [ ]:
println("--------------------------------- Running CUDA kNN ---------------------------------")
@time nearest_neighbors(FusionAGG)
display(FusionAGG.Simulation.Neighbor.idx_red)

In [ ]:
println("-------------------------------- Running CUDA Force --------------------------------")
@time cu_force(FusionAGG,Model,1.0)
display(FusionAGG.Position)

In [ ]:
model = Model
agg = FusionAGG

t = 0.0
for t = 0.0:model.Time.dt:model.Time.tₛᵢₘ
    global position_ant
    
    position_ant = copy(agg.Position)

    # # Saving data in a given time (nₛₐᵥₑ)
    if (Int(t/model.Time.dt) % model.Time.nₛₐᵥₑ+1) == 1
        pos = Matrix(agg.Position)
        open(full_path(model.Output.path_output,model.Output.name_output)*".xyz", "a") do f
            write(f, "$(size(agg.Position, 1))\n")
            write(f, "t=$(t)\n")
            writedlm(f,hcat(agg.Geometry.outline,pos), ' ')
        end
    end

    # # Calculating the kNN for the aggregate
    # if (step % model.Time.nₖₙₙ+1) == 1
    #     nearest_neighbors(agg)
    # end

    # # Compute the forces between each pair of particles in `agg` and their displacement.
    t_nₖₙₙ = Int(t/model.Time.dt) % model.Time.nₖₙₙ+1
    cu_force(agg,model,t_nₖₙₙ)


    if any(isnan.(agg.Position)) & any(.!isfinite.(agg.Position)) == true
        break
    end
end

println("t = $(t) | step = $(Int(t/model.Time.dt))")

println(any(isnan.(position_ant)) & any(.!isfinite.(position_ant))) 
display(position_ant)

println(any(isnan.(agg.Position)) & any(.!isfinite.(agg.Position)))
display(agg.Position)

In [ ]:
Points = position_ant
Idx_red = FusionAGG.Simulation.Neighbor.idx_red

threads =(16,3)
blocks  =(cld.(size(Points,1)+1,threads[1]),1)

# Running GPU for calculing force
@cuda threads=threads blocks=blocks sum_force!(
    Idx_red,
    Points,
    FusionAGG.Simulation.Force.F,
    FusionAGG.Simulation.Parameter.Force,
    model.Time.dt
)

Points

# later

In [ ]:
# using ProgressMeter

# # function simulate(agg::Aggregate, model::ModelSet, saving::Bool)
# model = Model
# agg = FusionAGG

# # Calculating position of every cell on the fusion
# @showprogress "Simulating Aggregate..." for step::Int = 0:1:(model.Time.tₛᵢₘ/model.Time.dt)
    
#     # # Saving data in a given time (nₛₐᵥₑ)
#     # if (step % model.Time.nₛₐᵥₑ+1) == 1
#         pos = Matrix(agg.Position)
#         open(full_path(model.Output.path_output,model.Output.name_output)*".xyz", "a") do f
#             write(f, "$(size(agg.Position, 1))\n")
#             write(f, "t=$(step*model.Time.dt)\n")
#             writedlm(f,hcat(agg.Geometry.outline,pos), ' ')
#         end
#     # end

#     # # Calculating the kNN for the aggregate
#     # if (step % model.Time.nₖₙₙ+1) == 1
#     #     nearest_neighbors(agg)
#     # end

#     # # Compute the forces between each pair of particles in `agg` and their displacement.
#     t_nₖₙₙ = step % model.Time.nₖₙₙ+1
#     cu_force(agg,model,t_nₖₙₙ)
# end